# import libraries

In [1]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=5e1f424366ceaed3c8af14673b0b83aa5ee324989208b116f0cc2c86e548e826
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [2]:
import os
import logging
import torch
import numpy as np
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass

from transformers import (
    DistilBertConfig,
    DistilBertForTokenClassification,
    DistilBertTokenizer,
    # AdamW,
    get_linear_schedule_with_warmup,
)
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from torch.nn import CrossEntropyLoss
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report
from tqdm import tqdm

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# initializations

In [3]:
@dataclass
class Config:
    """Configuration for DistilBERT model training"""
    model_name: str = "distilbert-base-cased"
    max_seq_length: int = 256
    batch_size: int = 8
    learning_rate: float = 5e-5
    num_epochs: int = 10
    weight_decay: float = 0.0
    warmup_steps: int = 0
    adam_epsilon: float = 1e-8
    max_grad_norm: float = 1.0
    seed: int = 42

# Utility functions

In [4]:
class InputExample:
    """A single training/test example for token classification"""

    def __init__(self, guid: str, words: List[str], labels: List[str]):
        self.guid = guid
        self.words = words
        self.labels = labels


class InputFeatures:
    """A single set of features of data"""

    def __init__(self, input_ids, attention_mask, label_ids):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.label_ids = label_ids



In [5]:
## Training

In [6]:
# class DistilBertNERTraining:
#     """
#     Class for training DistilBERT-based NER models
#     DistilBERT is 40% smaller and 60% faster than BERT while retaining 97% of performance
#     """

#     def __init__(
#         self,
#         labels: List[str],
#         model_save_dir: str,
#         config: Optional[DistilBertConfig] = None,
#         device: Optional[str] = None
#     ):
#         """
#         Initialize the training class

#         Args:
#             labels: List of NER labels (e.g., ['O', 'Authentication'])
#             model_save_dir: Directory to save the trained model
#             config: Training configuration
#             device: Device to use for training ('cuda' or 'cpu')
#         """
#         self.labels = labels
#         self.label_map = {label: i for i, label in enumerate(labels)}
#         self.num_labels = len(labels)
#         self.model_save_dir = model_save_dir
#         self.config = config or DistilBertConfig()

#         # Set device
#         if device:
#             self.device = torch.device(device)
#         else:
#             self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#         # Padding label ID
#         self.pad_token_label_id = CrossEntropyLoss().ignore_index

#         # Initialize model and tokenizer as None
#         self.model = None
#         self.tokenizer = None

#         # Set random seed for reproducibility
#         self._set_seed()

#         logger.info(f"Initialized DistilBertNERTraining with {self.num_labels} labels")
#         logger.info(f"Device: {self.device}")
#         logger.info(f"Model: {self.config.model_name}")

#     def _set_seed(self):
#         """Set random seed for reproducibility"""
#         import random
#         random.seed(self.config.seed)
#         np.random.seed(self.config.seed)
#         torch.manual_seed(self.config.seed)
#         if torch.cuda.is_available():
#             torch.cuda.manual_seed_all(self.config.seed)

#     def load_pretrained_model(self):
#         """Load pretrained DistilBERT model and tokenizer"""
#         logger.info(f"Loading pretrained DistilBERT model: {self.config.model_name}")

#         # Load tokenizer
#         self.tokenizer = DistilBertTokenizer.from_pretrained(
#             self.config.model_name,
#             do_lower_case=False
#         )

#         # Load model configuration
#         model_config = DistilBertConfig.from_pretrained(
#             self.config.model_name,
#             num_labels=self.num_labels
#         )

#         # Load model
#         self.model = DistilBertForTokenClassification.from_pretrained(
#             self.config.model_name,
#             config=model_config
#         )

#         self.model.to(self.device)
#         logger.info("DistilBERT model loaded successfully")

#     def _read_examples_from_file(self, file_path: str, mode: str = "train") -> List[InputExample]:
#         """
#         Read examples from a CoNLL-format file

#         Args:
#             file_path: Path to the input file
#             mode: Mode indicator (train/dev/test)

#         Returns:
#             List of InputExample objects
#         """
#         examples = []
#         guid_index = 1

#         with open(file_path, 'r', encoding='utf-8') as f:
#             words = []
#             labels = []

#             for line in f:
#                 line = line.strip()

#                 if line.startswith("-DOCSTART-") or line == "":
#                     if words:
#                         examples.append(InputExample(
#                             guid=f"{mode}-{guid_index}",
#                             words=words,
#                             labels=labels
#                         ))
#                         guid_index += 1
#                         words = []
#                         labels = []
#                 else:
#                     splits = line.split()
#                     if len(splits) >= 2:
#                         words.append(splits[0])
#                         labels.append(splits[-1])

#             # Add last example if exists
#             if words:
#                 examples.append(InputExample(
#                     guid=f"{mode}-{guid_index}",
#                     words=words,
#                     labels=labels
#                 ))

#         logger.info(f"Read {len(examples)} examples from {file_path}")
#         return examples

#     def _convert_examples_to_features(
#         self,
#         examples: List[InputExample]
#     ) -> List[InputFeatures]:
#         """
#         Convert examples to features suitable for DistilBERT
#         Note: DistilBERT doesn't use token_type_ids

#         Args:
#             examples: List of InputExample objects

#         Returns:
#             List of InputFeatures objects
#         """
#         features = []

#         for ex_index, example in enumerate(tqdm(examples, desc="Converting examples")):
#             tokens = []
#             label_ids = []

#             # Tokenize each word and align labels
#             for word, label in zip(example.words, example.labels):
#                 word_tokens = self.tokenizer.tokenize(word)

#                 if len(word_tokens) > 0:
#                     tokens.extend(word_tokens)
#                     # Use the real label for the first token, pad for others
#                     label_ids.extend([self.label_map[label]] +
#                                    [self.pad_token_label_id] * (len(word_tokens) - 1))

#             # Truncate if necessary
#             max_length = self.config.max_seq_length - 2  # Account for [CLS] and [SEP]
#             if len(tokens) > max_length:
#                 tokens = tokens[:max_length]
#                 label_ids = label_ids[:max_length]

#             # Add special tokens
#             tokens = [self.tokenizer.cls_token] + tokens + [self.tokenizer.sep_token]
#             label_ids = [self.pad_token_label_id] + label_ids + [self.pad_token_label_id]

#             # Convert tokens to IDs
#             input_ids = self.tokenizer.convert_tokens_to_ids(tokens)

#             # Create attention mask
#             attention_mask = [1] * len(input_ids)

#             # Pad to max length
#             padding_length = self.config.max_seq_length - len(input_ids)
#             input_ids += [self.tokenizer.pad_token_id] * padding_length
#             attention_mask += [0] * padding_length
#             label_ids += [self.pad_token_label_id] * padding_length

#             assert len(input_ids) == self.config.max_seq_length
#             assert len(attention_mask) == self.config.max_seq_length
#             assert len(label_ids) == self.config.max_seq_length

#             features.append(InputFeatures(
#                 input_ids=input_ids,
#                 attention_mask=attention_mask,
#                 label_ids=label_ids
#             ))

#         return features

#     def _create_dataloader(
#         self,
#         features: List[InputFeatures],
#         shuffle: bool = True
#     ) -> DataLoader:
#         """
#         Create a DataLoader from features

#         Args:
#             features: List of InputFeatures
#             shuffle: Whether to shuffle the data

#         Returns:
#             DataLoader object
#         """
#         all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
#         all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
#         all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)

#         dataset = TensorDataset(all_input_ids, all_attention_mask, all_label_ids)

#         sampler = RandomSampler(dataset) if shuffle else SequentialSampler(dataset)
#         dataloader = DataLoader(dataset, sampler=sampler, batch_size=self.config.batch_size)

#         return dataloader

#     def train(self, train_file: str, dev_file: Optional[str] = None):
#         """
#         Train the model

#         Args:
#             train_file: Path to training data file
#             dev_file: Optional path to development/validation data file
#         """
#         if self.model is None or self.tokenizer is None:
#             raise ValueError("Model not loaded. Call load_pretrained_model() first.")

#         logger.info("Starting training...")

#         # Load and prepare training data
#         train_examples = self._read_examples_from_file(train_file, mode="train")
#         train_features = self._convert_examples_to_features(train_examples)
#         train_dataloader = self._create_dataloader(train_features, shuffle=True)

#         # Calculate total training steps
#         num_train_steps = len(train_dataloader) * self.config.num_epochs

#         # Prepare optimizer and scheduler
#         no_decay = ["bias", "LayerNorm.weight"]
#         optimizer_grouped_parameters = [
#             {
#                 "params": [p for n, p in self.model.named_parameters()
#                           if not any(nd in n for nd in no_decay)],
#                 "weight_decay": self.config.weight_decay,
#             },
#             {
#                 "params": [p for n, p in self.model.named_parameters()
#                           if any(nd in n for nd in no_decay)],
#                 "weight_decay": 0.0,
#             },
#         ]

#         optimizer = AdamW(
#             optimizer_grouped_parameters,
#             lr=self.config.learning_rate,
#             eps=self.config.adam_epsilon
#         )

#         scheduler = get_linear_schedule_with_warmup(
#             optimizer,
#             num_warmup_steps=self.config.warmup_steps,
#             num_training_steps=num_train_steps
#         )

#         # Training loop
#         logger.info(f"***** Running training *****")
#         logger.info(f"  Num examples = {len(train_examples)}")
#         logger.info(f"  Num epochs = {self.config.num_epochs}")
#         logger.info(f"  Batch size = {self.config.batch_size}")
#         logger.info(f"  Total optimization steps = {num_train_steps}")

#         global_step = 0
#         train_loss = 0.0

#         self.model.zero_grad()

#         for epoch in range(self.config.num_epochs):
#             logger.info(f"Epoch {epoch + 1}/{self.config.num_epochs}")

#             self.model.train()
#             epoch_loss = 0.0

#             for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
#                 batch = tuple(t.to(self.device) for t in batch)

#                 inputs = {
#                     "input_ids": batch[0],
#                     "attention_mask": batch[1],
#                     "labels": batch[2]
#                 }

#                 outputs = self.model(**inputs)
#                 loss = outputs[0]

#                 loss.backward()

#                 train_loss += loss.item()
#                 epoch_loss += loss.item()

#                 torch.nn.utils.clip_grad_norm_(
#                     self.model.parameters(),
#                     self.config.max_grad_norm
#                 )

#                 optimizer.step()
#                 scheduler.step()
#                 self.model.zero_grad()
#                 global_step += 1

#             avg_epoch_loss = epoch_loss / len(train_dataloader)
#             logger.info(f"Epoch {epoch + 1} - Average loss: {avg_epoch_loss:.4f}")

#             # Evaluate on dev set if provided
#             if dev_file:
#                 dev_metrics = self.evaluate(dev_file)
#                 logger.info(f"Dev metrics - Precision: {dev_metrics['precision']:.4f}, "
#                           f"Recall: {dev_metrics['recall']:.4f}, F1: {dev_metrics['f1']:.4f}")

#         avg_train_loss = train_loss / global_step
#         logger.info(f"Training completed. Average training loss: {avg_train_loss:.4f}")

#     def evaluate(self, eval_file: str) -> Dict[str, float]:
#         """
#         Evaluate the model

#         Args:
#             eval_file: Path to evaluation data file

#         Returns:
#             Dictionary containing evaluation metrics
#         """
#         if self.model is None or self.tokenizer is None:
#             raise ValueError("Model not loaded.")

#         logger.info("Running evaluation...")

#         # Load and prepare evaluation data
#         eval_examples = self._read_examples_from_file(eval_file, mode="eval")
#         eval_features = self._convert_examples_to_features(eval_examples)
#         eval_dataloader = self._create_dataloader(eval_features, shuffle=False)

#         # Evaluation
#         self.model.eval()
#         eval_loss = 0.0
#         nb_eval_steps = 0
#         preds = None
#         out_label_ids = None

#         for batch in tqdm(eval_dataloader, desc="Evaluating"):
#             batch = tuple(t.to(self.device) for t in batch)

#             with torch.no_grad():
#                 inputs = {
#                     "input_ids": batch[0],
#                     "attention_mask": batch[1],
#                     "labels": batch[2]
#                 }

#                 outputs = self.model(**inputs)
#                 tmp_eval_loss, logits = outputs[:2]

#                 eval_loss += tmp_eval_loss.item()

#             nb_eval_steps += 1

#             if preds is None:
#                 preds = logits.detach().cpu().numpy()
#                 out_label_ids = inputs["labels"].detach().cpu().numpy()
#             else:
#                 preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
#                 out_label_ids = np.append(
#                     out_label_ids,
#                     inputs["labels"].detach().cpu().numpy(),
#                     axis=0
#                 )

#         eval_loss = eval_loss / nb_eval_steps
#         preds = np.argmax(preds, axis=2)

#         # Convert IDs to labels
#         id_to_label = {i: label for label, i in self.label_map.items()}

#         out_label_list = []
#         preds_list = []

#         for i in range(out_label_ids.shape[0]):
#             temp_out = []
#             temp_pred = []
#             for j in range(out_label_ids.shape[1]):
#                 if out_label_ids[i, j] != self.pad_token_label_id:
#                     temp_out.append(id_to_label[out_label_ids[i][j]])
#                     temp_pred.append(id_to_label[preds[i][j]])
#             out_label_list.append(temp_out)
#             preds_list.append(temp_pred)

#         # Calculate metrics
#         results = {
#             "loss": eval_loss,
#             "precision": precision_score(out_label_list, preds_list),
#             "recall": recall_score(out_label_list, preds_list),
#             "f1": f1_score(out_label_list, preds_list),
#         }

#         # Print detailed classification report
#         logger.info("\nClassification Report:")
#         logger.info("\n" + classification_report(out_label_list, preds_list))

#         return results

#     def save_model(self):
#         """Save the trained model and tokenizer"""
#         logger.info(f"Saving model to {self.model_save_dir}")

#         if not os.path.exists(self.model_save_dir):
#             os.makedirs(self.model_save_dir)

#         # Save model
#         model_to_save = self.model.module if hasattr(self.model, 'module') else self.model
#         model_to_save.save_pretrained(self.model_save_dir)

#         # Save tokenizer
#         self.tokenizer.save_pretrained(self.model_save_dir)

#         # Save labels
#         labels_file = os.path.join(self.model_save_dir, "labels.txt")
#         with open(labels_file, 'w') as f:
#             f.write('\n'.join(self.labels))

#         logger.info("Model saved successfully")


In [17]:
class DistilBertNERTraining:
    """
    Class for training DistilBERT-based NER models
    DistilBERT is 40% smaller and 60% faster than BERT while retaining 97% of performance
    """

    def __init__(
        self,
        labels: List[str],
        model_save_dir: str,
        config: Optional[DistilBertConfig] = None,
        device: Optional[str] = None
    ):
        """
        Initialize the training class

        Args:
            labels: List of NER labels (e.g., ['O', 'Authentication'])
            model_save_dir: Directory to save the trained model
            config: Training configuration
            device: Device to use for training ('cuda' or 'cpu')
        """
        self.labels = labels
        self.label_map = {label: i for i, label in enumerate(labels)}
        self.num_labels = len(labels)
        self.model_save_dir = model_save_dir
        self.config = config or DistilBertConfig()

        # Set device
        if device:
            self.device = torch.device(device)
        else:
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Padding label ID
        self.pad_token_label_id = CrossEntropyLoss().ignore_index

        # Initialize model and tokenizer as None
        self.model = None
        self.tokenizer = None

        # Set random seed for reproducibility
        self._set_seed()

        logger.info(f"Initialized DistilBertNERTraining with {self.num_labels} labels")
        logger.info(f"Device: {self.device}")
        logger.info(f"Model: {self.config.model_name}")

    def _set_seed(self):
        """Set random seed for reproducibility"""
        import random
        random.seed(self.config.seed)
        np.random.seed(self.config.seed)
        torch.manual_seed(self.config.seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(self.config.seed)

    def load_pretrained_model(self):
        """Load pretrained DistilBERT model and tokenizer"""
        logger.info(f"Loading pretrained DistilBERT model: {self.config.model_name}")

        # Load tokenizer
        self.tokenizer = DistilBertTokenizer.from_pretrained(
            self.config.model_name,
            do_lower_case=False
        )

        # Load model configuration
        model_config = DistilBertConfig.from_pretrained(
            self.config.model_name,
            num_labels=self.num_labels
        )

        # Load model
        self.model = DistilBertForTokenClassification.from_pretrained(
            self.config.model_name,
            config=model_config
        )

        self.model.to(self.device)
        logger.info("DistilBERT model loaded successfully")

    def _read_examples_from_file(self, file_path: str, mode: str = "train") -> List[InputExample]:
        """
        Read examples from a CoNLL-format file

        Args:
            file_path: Path to the input file
            mode: Mode indicator (train/dev/test)

        Returns:
            List of InputExample objects
        """
        examples = []
        guid_index = 1

        with open(file_path, 'r', encoding='utf-8') as f:
            words = []
            labels = []

            for line in f:
                line = line.strip()

                if line.startswith("-DOCSTART-") or line == "":
                    if words:
                        examples.append(InputExample(
                            guid=f"{mode}-{guid_index}",
                            words=words,
                            labels=labels
                        ))
                        guid_index += 1
                        words = []
                        labels = []
                else:
                    splits = line.split()
                    if len(splits) >= 2:
                        words.append(splits[0])
                        labels.append(splits[-1])

            # Add last example if exists
            if words:
                examples.append(InputExample(
                    guid=f"{mode}-{guid_index}",
                    words=words,
                    labels=labels
                ))

        logger.info(f"Read {len(examples)} examples from {file_path}")
        return examples

    def _convert_examples_to_features(
        self,
        examples: List[InputExample]
    ) -> List[InputFeatures]:
        """
        Convert examples to features suitable for DistilBERT
        Note: DistilBERT doesn't use token_type_ids

        Args:
            examples: List of InputExample objects

        Returns:
            List of InputFeatures objects
        """
        features = []

        for ex_index, example in enumerate(tqdm(examples, desc="Converting examples")):
            tokens = []
            label_ids = []

            # Tokenize each word and align labels
            for word, label in zip(example.words, example.labels):
                word_tokens = self.tokenizer.tokenize(word)

                if len(word_tokens) > 0:
                    tokens.extend(word_tokens)
                    # Use the real label for the first token, pad for others
                    label_ids.extend([self.label_map[label]] +
                                   [self.pad_token_label_id] * (len(word_tokens) - 1))

            # Truncate if necessary
            max_length = self.config.max_seq_length - 2  # Account for [CLS] and [SEP]
            if len(tokens) > max_length:
                tokens = tokens[:max_length]
                label_ids = label_ids[:max_length]

            # Add special tokens
            tokens = [self.tokenizer.cls_token] + tokens + [self.tokenizer.sep_token]
            label_ids = [self.pad_token_label_id] + label_ids + [self.pad_token_label_id]

            # Convert tokens to IDs
            input_ids = self.tokenizer.convert_tokens_to_ids(tokens)

            # Create attention mask
            attention_mask = [1] * len(input_ids)

            # Pad to max length
            padding_length = self.config.max_seq_length - len(input_ids)
            input_ids += [self.tokenizer.pad_token_id] * padding_length
            attention_mask += [0] * padding_length
            label_ids += [self.pad_token_label_id] * padding_length

            assert len(input_ids) == self.config.max_seq_length
            assert len(attention_mask) == self.config.max_seq_length
            assert len(label_ids) == self.config.max_seq_length

            features.append(InputFeatures(
                input_ids=input_ids,
                attention_mask=attention_mask,
                label_ids=label_ids
            ))

        return features

    def _create_dataloader(
        self,
        features: List[InputFeatures],
        shuffle: bool = False
    ) -> DataLoader:
        """
        Create a DataLoader from features

        Args:
            features: List of InputFeatures
            shuffle: Whether to shuffle the data

        Returns:
            DataLoader object
        """
        all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
        all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_ids for f in features], dtype=torch.long)

        dataset = TensorDataset(all_input_ids, all_attention_mask, all_label_ids)

        sampler = RandomSampler(dataset) if shuffle else SequentialSampler(dataset)
        dataloader = DataLoader(dataset, sampler=sampler, batch_size=self.config.batch_size)

        return dataloader

    def train(self, train_file: str, dev_file: Optional[str] = None):
        """
        Train the model

        Args:
            train_file: Path to training data file
            dev_file: Optional path to development/validation data file
        """
        if self.model is None or self.tokenizer is None:
            raise ValueError("Model not loaded. Call load_pretrained_model() first.")

        logger.info("Starting training...")

        # Load and prepare training data
        train_examples = self._read_examples_from_file(train_file, mode="train")
        train_features = self._convert_examples_to_features(train_examples)
        train_dataloader = self._create_dataloader(train_features, shuffle=True)

        # Calculate total training steps
        num_train_steps = len(train_dataloader) * self.config.num_epochs

        # Prepare optimizer and scheduler
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in self.model.named_parameters()
                          if not any(nd in n for nd in no_decay)],
                "weight_decay": self.config.weight_decay,
            },
            {
                "params": [p for n, p in self.model.named_parameters()
                          if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]

        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=self.config.learning_rate,
            eps=self.config.adam_epsilon
        )

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=self.config.warmup_steps,
            num_training_steps=num_train_steps
        )

        # Training loop
        logger.info(f"***** Running training *****")
        logger.info(f"  Num examples = {len(train_examples)}")
        logger.info(f"  Num epochs = {self.config.num_epochs}")
        logger.info(f"  Batch size = {self.config.batch_size}")
        logger.info(f"  Total optimization steps = {num_train_steps}")

        global_step = 0
        train_loss = 0.0

        self.model.zero_grad()

        for epoch in range(self.config.num_epochs):
            logger.info(f"Epoch {epoch + 1}/{self.config.num_epochs}")

            self.model.train()
            epoch_loss = 0.0

            for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
                batch = tuple(t.to(self.device) for t in batch)

                inputs = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    "labels": batch[2]
                }

                outputs = self.model(**inputs)
                loss = outputs[0]

                loss.backward()

                train_loss += loss.item()
                epoch_loss += loss.item()

                torch.nn.utils.clip_grad_norm_(
                    self.model.parameters(),
                    self.config.max_grad_norm
                )

                optimizer.step()
                scheduler.step()
                self.model.zero_grad()
                global_step += 1

            avg_epoch_loss = epoch_loss / len(train_dataloader)
            logger.info(f"Epoch {epoch + 1} - Average loss: {avg_epoch_loss:.4f}")

            # Evaluate on dev set if provided
            if dev_file:
                dev_metrics, _ = self.evaluate(dev_file, return_predictions=False)
                logger.info(f"Dev metrics - Precision: {dev_metrics['precision']:.4f}, "
                          f"Recall: {dev_metrics['recall']:.4f}, F1: {dev_metrics['f1']:.4f}")

        avg_train_loss = train_loss / global_step
        logger.info(f"Training completed. Average training loss: {avg_train_loss:.4f}")

    def evaluate(self, eval_file: str, return_predictions: bool = True) -> Tuple[Dict[str, float], Optional[str]]:
        """
        Evaluate the model

        Args:
            eval_file: Path to evaluation data file
            return_predictions: If True, return predictions text along with metrics

        Returns:
            Tuple of (metrics_dict, predictions_text)
            - metrics_dict: Dictionary containing evaluation metrics
            - predictions_text: String with "token prediction" format (None if return_predictions=False)
        """
        if self.model is None or self.tokenizer is None:
            raise ValueError("Model not loaded.")

        logger.info("Running evaluation...")

        # Load and prepare evaluation data
        eval_examples = self._read_examples_from_file(eval_file, mode="eval")
        eval_features = self._convert_examples_to_features(eval_examples)
        eval_dataloader = self._create_dataloader(eval_features, shuffle=False)

        # Evaluation
        self.model.eval()
        eval_loss = 0.0
        nb_eval_steps = 0
        preds = None
        out_label_ids = None

        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            batch = tuple(t.to(self.device) for t in batch)

            with torch.no_grad():
                inputs = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    "labels": batch[2]
                }

                outputs = self.model(**inputs)
                tmp_eval_loss, logits = outputs[:2]

                eval_loss += tmp_eval_loss.item()

            nb_eval_steps += 1

            if preds is None:
                preds = logits.detach().cpu().numpy()
                out_label_ids = inputs["labels"].detach().cpu().numpy()
            else:
                preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
                out_label_ids = np.append(
                    out_label_ids,
                    inputs["labels"].detach().cpu().numpy(),
                    axis=0
                )

        eval_loss = eval_loss / nb_eval_steps
        preds = np.argmax(preds, axis=2)

        # Convert IDs to labels
        id_to_label = {i: label for label, i in self.label_map.items()}

        out_label_list = []
        preds_list = []

        for i in range(out_label_ids.shape[0]):
            temp_out = []
            temp_pred = []
            for j in range(out_label_ids.shape[1]):
                if out_label_ids[i, j] != self.pad_token_label_id:
                    temp_out.append(id_to_label[out_label_ids[i][j]])
                    temp_pred.append(id_to_label[preds[i][j]])
            out_label_list.append(temp_out)
            preds_list.append(temp_pred)

        # Calculate metrics
        results = {
            "loss": eval_loss,
            "precision": precision_score(out_label_list, preds_list),
            "recall": recall_score(out_label_list, preds_list),
            "f1": f1_score(out_label_list, preds_list),
        }

        # Print detailed classification report
        logger.info("\nClassification Report:")
        logger.info("\n" + classification_report(out_label_list, preds_list))

        # Generate predictions text if requested
        predictions_text = None
        if return_predictions:
            # Read original tokens from file
            original_tokens = []
            with open(eval_file, 'r', encoding='utf-8') as f:
                for line in f:
                    line = line.strip()
                    if line.startswith("-DOCSTART-") or line == "":
                        if original_tokens:
                            original_tokens.append("")  # Empty line between sentences
                    else:
                        splits = line.split()
                        if len(splits) >= 1:
                            original_tokens.append(splits[0])

            # Flatten predictions list
            flat_predictions = []
            for sent_preds in preds_list:
                flat_predictions.extend(sent_preds)
                flat_predictions.append("")  # Empty line between sentences

            # Create predictions text
            predictions_lines = []
            token_idx = 0
            pred_idx = 0

            for token in original_tokens:
                if token == "":
                    predictions_lines.append("")
                else:
                    if pred_idx < len(flat_predictions) and flat_predictions[pred_idx] != "":
                        predictions_lines.append(f"{token} {flat_predictions[pred_idx]}")
                        pred_idx += 1
                    else:
                        predictions_lines.append(f"{token} O")
                        if pred_idx < len(flat_predictions):
                            pred_idx += 1

            predictions_text = "\n".join(predictions_lines)

        return results, predictions_text

    def save_model(self):
        """Save the trained model and tokenizer"""
        logger.info(f"Saving model to {self.model_save_dir}")

        if not os.path.exists(self.model_save_dir):
            os.makedirs(self.model_save_dir)

        # # Save model
        # model_to_save = self.model.module if hasattr(self.model, 'module') else self.model
        # model_to_save.save_pretrained(self.model_save_dir)

        # Save tokenizer
        self.tokenizer.save_pretrained(self.model_save_dir)
        self.model.save_pretrained(self.model_save_dir)

        # Save labels
        labels_file = os.path.join(self.model_save_dir, "labels.txt")
        with open(labels_file, 'w') as f:
            f.write('\n'.join(self.labels))

        logger.info("Model saved successfully")




# Training

In [8]:
labels = ['O', 'Authentication']


In [9]:
config = Config(
    model_name="distilbert-base-cased",
    max_seq_length=256,
    batch_size=8,
    num_epochs=5,
    learning_rate=5e-5
)

In [10]:
ls drive/MyDrive/exl/Authentication/data

pred_v0.txt  test.txt  train.txt


In [11]:
trainer = DistilBertNERTraining(
    labels=labels,
    model_save_dir="drive/MyDrive/exl/Authentication/model",
    config=config
)



In [12]:
# Load pretrained model
trainer.load_pretrained_model()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Train
trainer.train(
    train_file="drive/MyDrive/exl/Authentication/data/train.txt"
    # dev_file="./data/dev.txt"  # Optional
)



Training: 100%|██████████| 25/25 [00:04<00:00,  5.85it/s]


In [19]:
trainer.model.save_pretrained("drive/MyDrive/exl/Authentication/model")

In [21]:
trainer.save_model()

In [15]:
# Evaluation

In [34]:
# Evaluate
eval_metrics, pred_text = trainer.evaluate("drive/MyDrive/exl/Authentication/data/test.txt", return_predictions=True)
print(f"\nEvaluation metrics: {eval_metrics}")

Evaluating: 100%|██████████| 7/7 [00:00<00:00, 18.18it/s]



Evaluation metrics: {'loss': 0.06532351996949208, 'precision': np.float64(0.8269230769230769), 'recall': np.float64(0.9148936170212766), 'f1': np.float64(0.8686868686868686)}


In [35]:
print(pred_text)

Agent: O
Maple O
Bank's O
Retirement O
Planning, O
this O
is O
Nicole O
speaking. O
How O
can O
I O
help? O
Customer: O
Hello O
Nicole, O
I'm O
55 O
and O
considering O
early O
retirement O
next O
year. O
Agent: O
That's O
a O
major O
life O
transition O
that O
requires O
careful O
planning. O
First, O
let's O
discuss O
your O
retirement O
accounts O
- O
do O
you O
have O
a O
401(k), O
IRA, O
or O
other O
savings? O
Customer: O
Yes, O
$800,000 O
in O
my O
401(k) O
and O
$200,000 O
in O
IRAs. O
Agent: O
At O
your O
age, O
you O
can O
access O
401(k) O
funds O
without O
penalty O
if O
you O
retire, O
but O
I'd O
recommend O
rolling O
it O
to O
an O
IRA O
first O
for O
more O
flexibility. O
We O
should O
analyze O
your O
Social O
Security O
strategy, O
healthcare O
coverage, O
and O
tax-efficient O
withdrawal O
plan. O
Would O
you O
like O
to O
schedule O
a O
comprehensive O
retirement O
review O
with O
our O
CFP®? O
Customer: O
Yes, O
what O
information O
should O
I O
prepare? O
Agent: O

In [36]:
with open("drive/MyDrive/exl/Authentication/data/pred_v0.txt",'w+') as f:
  f.write(pred_text)